# Práctica 04: Algoritmo A*

El laberinto está representado en una matriz, donde:

- 0 es un espacio libre
- 1 es un obstáculo
- E es la entrada del laberinto
- S es la salida del laberinto

Para esta implementación se usará la explicación dada en clase de laboratorio

### Sección i) - Creación de listas abierta y cerrada

```
lista_abierta = []  
lista_cerrada = []  
```

 La lista abierta almacena las casillas que aún no se han explorado, mientras que la lista cerrada almacena las casillas que ya se han examinado.

### Sección ii) - Inicialización del nodo de inicio y su colocación en la lista abierta

```
nodo_inicio = Nodo(inicio[0], inicio[1])  
abierta.append(nodo_inicio)  
```

La casilla de entrada se agrega a la lista abierta con su valor (f) inicializado en cero. Esto se hace al inicio de la función ``` buscar_camino_A_estrella```.

### Sección iii) Mientras la lista abierta no esté vacía:
<pre>
* a) Se busca el nodo con el menor valor de f en la lista abierta.
* b) Se elimina ese nodo de la lista abierta y se agrega a la lista cerrada.
* c) Se identifican los vecinos válidos del nodo actual.
* d) Para cada vecino:
       1. Si el vecino es la casilla objetivo, se termina.
       2. Se calculan los valores g, h y f para el vecino.
       3. Se verifica si el vecino ya está en la lista abierta y si tiene un valor de f menor. 
              Si es así, se ignora.
       4. Se verifica si el vecino ya está en la lista cerrada y si tiene un valor de f menor. 
              Si es así, se ignora. De lo contrario, se agrega el vecino a la lista abierta.
</pre>

<center><h1>INICIO DEL CÓDIGO</h1></center>


In [2]:
class Agente:
    def __init__(self, posicion):
        self.posicion = posicion  
        self.padre = None  
        self.g = 0  # Costo real del inicio al nodo actual
        self.h = 0  # Heurística (la distancia Manhattan)
        self.f = 0  # Costo total estimado desde el inicio al objetivo pasando por el nodo actual

    def mover(self, direccion, laberinto):
        x, y = self.posicion
        if direccion == "arriba" and x > 0:
            return [x - 1, y]
        elif direccion == "abajo" and x < len(laberinto) - 1:
            return [x + 1, y]
        elif direccion == "izquierda" and y > 0:
            return [x, y - 1]
        elif direccion == "derecha" and y < len(laberinto[0]) - 1:
            return [x, y + 1]
        else:
            return None

def obtener_vecinos_validos(casilla_actual, laberinto):
    """
    Esta función obtiene los vecinos válidos del agente en el laberinto.

    recibe:
        casilla_actual (Agente): El agente cuya posición se evalúa.
        laberinto (list): El laberinto representado como una matriz.

    devuelve:
        list: Una lista de objetos Agente representando los vecinos válidos.
    """
    vecinos = []
    if casilla_actual is not None:
        for direccion in ["arriba", "abajo", "izquierda", "derecha"]:
            nueva_posicion = casilla_actual.mover(direccion, laberinto)
            if nueva_posicion and laberinto[nueva_posicion[0]][nueva_posicion[1]] != 1:
                vecino = Agente(nueva_posicion)
                vecinos.append(vecino)
    return vecinos

def reconstruir_camino(casilla_actual, laberinto):
    """
    Esta función reconstruye el camino desde la casilla actual hasta la posición inicial del agente.

    recibe:
        casilla_actual (Agente): La casilla actual.
        laberinto (list): El laberinto representado como una matriz.

    devuelve:
        list: Una lista que representa el camino desde la casilla actual hasta la posición inicial del agente.
    """
    camino = []
    while casilla_actual:
        camino.append(casilla_actual.posicion)
        casilla_actual = casilla_actual.padre

    camino = list(reversed(camino))
    print("Ruta seguida:")
    for posicion in camino:
        print(f"[{posicion[0]}, {posicion[1]}]")
    print("Se encontró la salida :D")  

    print("\nCamino encontrado:")
    for i in range(len(laberinto)):
        fila = ""
        for j in range(len(laberinto[0])):
            if [i, j] in camino:
                fila += "X "
            else:
                fila += "- "
        print(fila)

    return camino

def distancia_manhattan(laberinto):
    """
    Calcula la distancia de Manhattan entre dos nodos.

    recibe:
        laberinto (list): El laberinto representado como una matriz.

    devuelve:
        int: La distancia de Manhattan entre los dos nodos.
    """
    nodo_actual = punto_de_partida(laberinto)
    nodo_objetivo = salida_laberinto(laberinto)

    x, y = nodo_actual
    w, z = nodo_objetivo
    
    distancia = abs(x - w) + abs(y - z)
    return distancia

def punto_de_partida(laberinto):
    """
    Encuentra la posición inical del agente en el laberinto

    Recibe:
        laberinto (list): El laberinto representado como una matriz.

    Devuelve:
        tupla: La posición(coordenadas) inicial del agente
    """
    try:
        for i in range(len(laberinto)):
            for j in range(len(laberinto[0])):
                if laberinto[i][j] == "E":  # Comprobamos si encontramos la salida
                    return (i, j)  # Devolvemos las coordenadas 
    except ValueError:
        print("No se encontró la salida del laberinto.")

def salida_laberinto(laberinto):
    """
    Encuentra la posición de la salida del laberinto

    Recibe:
        laberinto (list): El laberinto representado como una matriz.

    Devuelve:
        tupla: La posición(coordenadas) de la salida del laberinto.
    """
    try:
        for i in range(len(laberinto)):
            for j in range(len(laberinto[0])):
                if laberinto[i][j] == "S":  # Comprobamos si encontramos la salida
                    return (i, j)  # Devolvemos las coordenadas 
    except ValueError:
        print("No se encontró la salida del laberinto.")

def buscar_camino_A_estrella(agente, laberinto):
    """
    Esta función implementa el algoritmo A* para encontrar el camino más corto entre la posición del agente y la salida en un laberinto.

    recibe:
        agente (Agente): El agente que navega por el laberinto.
        laberinto (list): El laberinto representado como una matriz.

    devuelve:
        list: Una lista que representa el camino más corto entre la posición del agente y la salida.
    """
    lista_abierta = [agente]
    lista_cerrada = []

    while lista_abierta:
        casilla_actual = min(lista_abierta, key=lambda casilla: casilla.f)
        lista_abierta.remove(casilla_actual)
        lista_cerrada.append(casilla_actual.posicion)  # Guardar solo la posición

        if laberinto[casilla_actual.posicion[0]][casilla_actual.posicion[1]] == "S":
            return reconstruir_camino(casilla_actual)
       
        vecinos = obtener_vecinos_validos(casilla_actual, laberinto)

        for vecino in vecinos:
            if vecino.posicion in lista_cerrada:  # Verificar si la posición ya está en la lista cerrada
                continue

            vecino.g = casilla_actual.g + 1
            vecino.h = distancia_manhattan(laberinto) 
            vecino.f = vecino.g + vecino.h

            if vecino in lista_abierta and vecino.f >= casilla_actual.f:
                continue

            vecino.padre = casilla_actual

            if vecino not in lista_abierta:
                lista_abierta.append(vecino)

    print("No se encontró un camino válido.")
    return []

# Representación del laberinto
laberinto = [
    ["E", 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 1, 1, 1],
    [0, 1, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 0, 0, 1, 0],
    [0, 0, 0, "S", 0, 0, 0, 0]
]

# Posición inicial del agente
agente = Agente([0, 0])

buscar_camino_A_estrella(agente, laberinto)


Ruta: [0, 1]
Ruta: [0, 2]
Ruta: [0, 3]
Ruta: [0, 4]
Ruta: [1, 4]
Ruta: [2, 4]
Ruta: [3, 4]
Ruta: [4, 4]
Ruta: [4, 5]
Ruta: [4, 6]
Ruta: [4, 7]
Ruta: [5, 7]
Ruta: [6, 7]
Ruta: [7, 7]
Ruta: [7, 6]
Ruta: [7, 5]
Ruta: [7, 4]
Ruta: [7, 3]
Se encontró la salida :D

Camino encontrado:
 X  X  X  X  X  -  -  - 
 -  -  -  -  X  -  -  - 
 -  -  -  -  X  -  -  - 
 -  -  -  -  X  -  -  - 
 -  -  -  -  X  X  X  X 
 -  -  -  -  -  -  -  X 
 -  -  -  -  -  -  -  X 
 -  -  -  X  X  X  X  X 
